In [1]:
import setup
setup.init_django()

In [2]:
import helpers.clients as helper_clients

In [10]:
from datetime import datetime, timedelta
days_ago = 60 # about 2 months
days_end = 30 # about 1 month

company_name = "Google"
company_ticker = "GOOG"
multiplier = 1

now = datetime.now()
from_date = (now - timedelta(days=days_ago)).strftime("%Y-%m-%d")
to_date = (now - timedelta(days=days_end)).strftime("%Y-%m-%d")

print(f"Syncing {company_ticker} from {from_date} to {to_date}")

Syncing GOOG from 2024-09-28 to 2024-10-28


In [11]:
client = helper_clients.PolygonAPIClient(
    ticker=company_ticker,
    multiplier=multiplier,
    from_date=from_date,
    to_date=to_date
)
dataset = client.get_stock_data()

Fetching data from https://api.polygon.io/v2/aggs/ticker/GOOG/range/1/minute/2024-09-28/2024-10-28?adjusted=True&sort=asc&limit=50000


In [12]:
len(dataset)

12005

In [13]:
new_stock_data = dataset[0]
new_stock_data

{'open_price': 165.37,
 'close_price': 165.4,
 'high_price': 165.4,
 'low_price': 165.37,
 'number_of_trades': 23,
 'volume': 489,
 'volume_weighted_average': 165.3756,
 'raw_timestamp': 1727683320000,
 'time': datetime.datetime(2024, 9, 30, 8, 2, tzinfo=<UTC>)}

In [14]:
from market.models import Company, StockQuote

In [15]:
company_obj, created = Company.objects.get_or_create(name=company_name, ticker=company_ticker)

In [16]:
StockQuote.objects.create(company=company_obj, **new_stock_data)

<StockQuote: StockQuote object (110427)>

In [10]:
# StockQuote.objects.bulk_create([StockQuote(company=company_obj, **new_stock_data)])

In [11]:
# new_quotes = []

# for data in dataset:
#     new_quotes.append(
#         StockQuote(company=company_obj, **data)
#     )

In [18]:
batch_size = 1000
for i in range(0, len(dataset), batch_size):
    batch_chunk = dataset[i:i+batch_size]
    chunked_quotes = []
    for data in batch_chunk:
        chunked_quotes.append(
            StockQuote(company=company_obj, **data)
        )
    StockQuote.objects.bulk_create(chunked_quotes, ignore_conflicts=True)

In [19]:
StockQuote.objects.all().count()

12005